In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import math
import yaml

with open('../params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

C:\Users\sobda\Anaconda2\envs\courseWork\lib\site-packages\numpy\__init__.py:148: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\sobda\Anaconda2\envs\courseWork\python.exe"
  * The NumPy version is: "1.21.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: Не найден указанный модуль.


# Preprocessing

In [103]:
raw_path = config['data_load']['dataset_csv']
train = pd.read_csv(os.path.join(raw_path, config['data_load']['train_csv']))
test = pd.read_csv(os.path.join(raw_path, config['data_load']['test_csv']))

In [104]:
train

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,...,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,...,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,...,06:00:00,0,0,0,54-103-006-01,0,0,0,0,0
951,54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:15:00,0,0,0,54-701-039-01,0,0,0,0,0
952,54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,...,08:00:00,1,0,0,54-002-014-01,1,0,1,0,0
953,54-501-022-01,Ж,в браке в настоящее время,европейская,Русские,Атеист / агностик,5 - ВУЗ,"представители законодат. органов власти,...",1,0,...,08:30:00,0,1,0,54-501-022-01,0,0,0,0,0


In [105]:
TARGET_COLS = config['preprocess_category']['TARGET_COLS']
ID_COL = config['preprocess_category']['ID_COL']
EDU_COL = config['preprocess_category']['EDU_COL']
SEX_COL = config['preprocess_category']['SEX_COL']
CAT_COLS = config['preprocess_category']['CAT_COLS']
OHE_COLS = config['preprocess_category']['OHE_COLS']
REAL_COLS = config['preprocess_category']['REAL_COLS']

In [106]:
def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df

In [107]:
def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df

In [108]:
def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
    df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
    df = df.drop(columns=[EDU_COL])
    return df

In [109]:
def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[SEX_COL].value_counts().index[0]
    df[SEX_COL] = df[SEX_COL].fillna(most_freq)
    return df

In [110]:
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')

    ohe_int_cols = df[OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
    return df
    

In [111]:
def delete_nan_value(df: pd.DataFrame) ->pd.DataFrame:
    ##Smoking
    period_smoke = list(df['Возраст курения'])
    number_of_сigarettes = list(df['Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        if row['Статус Курения'] == 'Никогда не курил(а)':
            if pd.isna(row['Возраст курения']):
                period_smoke[count] = 0
            if pd.isna(row['Сигарет в день']):
                number_of_сigarettes[count] = 0
        count += 1
    df = df.drop(columns=['Возраст курения', 'Сигарет в день'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст курения'] = period_smoke[count]
        df.loc[index,'Сигарет в день'] = number_of_сigarettes[count]
        count += 1 
    df = df[(np.isnan(df['Возраст курения'])) == False] 
    df['Возраст курения'] = df['Возраст курения'].astype(np.int8)
    df = df[(np.isnan(df['Сигарет в день'])) == False] 
    df['Сигарет в день'] = df['Сигарет в день'].astype(np.int8)
    

    ## Пассивное курение 
    frequency_second_hand_smoke = list(df['Частота пасс кур'])
    count = 0
    for index, row in df.iterrows():
        if row['Пассивное курение'] == 0:
            if pd.isna(row['Частота пасс кур']):
                frequency_second_hand_smoke[count] = 'менее 1 раза в день'
        count += 1
    df = df.drop(columns=['Частота пасс кур'])
    df.insert(21, 'Частота пасс кур', frequency_second_hand_smoke, True)
    df = df[(pd.isna(df['Частота пасс кур'])) == False] 
    df['Частота пасс кур'] = df['Частота пасс кур'].astype('category')
    
    ## алкоголь
    drinking_period = list(df['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        if row['Алкоголь'] == 'никогда не употреблял':
            if pd.isna(row['Возраст алког']):
                drinking_period[count] = 0
        count += 1
    df = df.drop(columns=['Возраст алког'])
    count = 0
    for index, row in df.iterrows():
        df.loc[index,'Возраст алког'] = drinking_period[count]
        count += 1
    df = df[(pd.isna(df['Возраст алког'])) == False] 
    df['Возраст алког'] = df['Возраст алког'].astype(np.int8)
    return df

In [112]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = cast_types(df)
    df = add_ord_edu(df)
    df = delete_nan_value(df)
    return df

In [113]:
def preprocess_target(df: pd.DataFrame) -> pd.DataFrame:
    df[TARGET_COLS] = df[TARGET_COLS].astype(np.int8)
    return df

def extract_target(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df, target = df.drop(TARGET_COLS, axis=1), df[TARGET_COLS]
    return df, target

In [114]:
import pickle
from typing import Union
from pandas import DataFrame
from pandas.core.indexes.base import Index as PandasIndex


def save_as_pickle(obj: Union[DataFrame, PandasIndex], path: str) -> None:
    if isinstance(obj, DataFrame):
        obj.to_pickle(path)
    elif isinstance(obj, PandasIndex):
        with open('path', 'wb') as f:
            pickle.dump(obj, f)

In [115]:
train = preprocess(train)
test = preprocess(test)


##только для train()
df, target = extract_target(train)
target = preprocess_target(target)
save_as_pickle(target, '../' + config['preprocess_paths']['processed_path'] + config['preprocess_paths']['target_data_train'])
save_as_pickle(df, '../' + config['preprocess_paths']['interim_path'] + config['preprocess_paths']['data_for_train'])
##только для test
save_as_pickle(test, '../' + config['preprocess_paths']['interim_path'] + config['preprocess_paths']['test_pkl'])

C:\Users\dsfs-\AppData\Local\Temp\ipykernel_8068\492307506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[TARGET_COLS] = df[TARGET_COLS].astype(np.int8)


In [116]:
target

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0
...,...,...,...,...,...
54-103-006-01,0,0,0,0,0
54-701-039-01,0,0,0,0,0
54-002-014-01,1,0,1,0,0


In [117]:
train.columns

Index(['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Профессия',
       'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни',
       'Сахарный диабет', 'Гепатит', 'Онкология',
       'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Пассивное курение', 'Частота пасс кур', 'Алкоголь',
       'Время засыпания', 'Время пробуждения', 'Сон после обеда',
       'Спорт, клубы', 'Религия, клубы', 'Артериальная гипертензия', 'ОНМК',
       'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность',
       'Прочие заболевания сердца', 'Образование_ord', 'Возраст курения',
       'Сигарет в день', 'Возраст алког'],
      dtype='object')

# Feature generation

 Из столбцов "Время пробуждения" и "Время засыпания" создадим два признака: 1. Является человек совой/Жаворонком/Голубем 2. Количество сна в часах

In [118]:
interim_data_path = '../data/interim/'
train = pd.read_pickle(os.path.join(interim_data_path, 'data_for_train.pkl'))
test = pd.read_pickle(os.path.join(interim_data_path, 'test.pkl'))

In [119]:
import datetime
def check_owl_or_lark(df: pd.DataFrame) -> pd.DataFrame:
    check_owl_or_lark = []
    for index, row in df.iterrows():
        if pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S' ) >= pd.to_datetime('23:00:00',format= '%H:%M:%S' ) or  pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S' ) < pd.to_datetime('5:00:00',format= '%H:%M:%S') :
            if pd.to_datetime(row['Время пробуждения'],format= '%H:%M:%S' ) >= pd.to_datetime('10:00:00',format= '%H:%M:%S' ):
                check_owl_or_lark.append('owl')
            else:
                check_owl_or_lark.append('pigeon')
        else:
            if pd.to_datetime(row['Время пробуждения'],format= '%H:%M:%S' ) < pd.to_datetime('10:00:00',format= '%H:%M:%S' ):
                 check_owl_or_lark.append('lark')
            else:
                check_owl_or_lark.append('pigeon')

    df.insert(25, 'Сова/Жаворонок/Голубь', check_owl_or_lark, True)
    df['Сова/Жаворонок/Голубь'] = df['Сова/Жаворонок/Голубь'].astype('category')
    return df   

In [120]:
def amount_of_sleep(df: pd.DataFrame) -> pd.DataFrame:
    amount_of_sleep = []
    for index, row in df.iterrows():
        time  = pd.to_datetime(row['Время пробуждения'], format= '%H:%M:%S') - pd.to_datetime(row['Время засыпания'],format= '%H:%M:%S')
        amount_of_sleep.append(int(time.to_pytimedelta().seconds/3600))
    
    df.insert(26, 'Часов сна', amount_of_sleep, True)
    df['Часов сна'] = df['Часов сна'].astype(np.int8)
    return df

    

In [121]:
def remove_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns=['Время засыпания', 'Время пробуждения'],axis = 1)
    return df

In [122]:
def feature_generation(df: pd.DataFrame) -> pd.DataFrame:
    df = check_owl_or_lark(df)
    df = amount_of_sleep(df)
    df = remove_unnecessary_columns(df)
    #df = cast_types(df)
    return df

In [123]:
train = feature_generation(train)
test = feature_generation(test)


save_as_pickle(test, '../data/processed/test.pkl')
save_as_pickle(train, '../data/processed/data_for_train.pkl')


# Modeling

In [124]:
import os
import pandas as pd
import numpy as np

from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *
#%pip install catboost

In [125]:
processed_data_path = '../data/processed/'
train = pd.read_pickle(os.path.join(processed_data_path, 'data_for_train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target_for_data_for_train.pkl'))

In [126]:
CATEGORIES_COL = ['Пол','Семья', 'Этнос', 'Национальность', 'Религия', 'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь', 'Сова/Жаворонок/Голубь']
pool_train = Pool(X_train, y_train,
                  cat_features = ['cut', 'color', 'clarity'])

In [127]:
target

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0
...,...,...,...,...,...
54-103-006-01,0,0,0,0,0
54-701-039-01,0,0,0,0,0
54-002-014-01,1,0,1,0,0


In [128]:
target_1 = target['Артериальная гипертензия']

In [129]:
target_1

ID
54-102-358-02    0
54-103-101-01    1
54-501-026-03    0
54-501-094-02    1
54-503-022-01    1
                ..
54-103-006-01    0
54-701-039-01    0
54-002-014-01    1
54-501-022-01    0
54-702-016-01    0
Name: Артериальная гипертензия, Length: 948, dtype: int8

In [130]:
RS = 77

In [131]:
X_train, X_test, Y_train, Y_test = train_test_split(train, target, train_size=0.8, random_state = RS)

In [132]:
import catboost
from catboost import Pool,cv
from catboost import CatBoostClassifier

In [136]:
pool = Pool(X_train, Y_train,
                  cat_features = CATEGORIES_COL)

In [155]:
model1 = CatBoostClassifier(iterations=1000, loss_function='MultiLogloss', eval_metric='MultiLogloss',
learning_rate=0.01, bootstrap_type='Bayesian', boost_from_average=False,
leaf_estimation_iterations=1, leaf_estimation_method='Gradient')

model1.fit(pool)

0:	learn: 0.6918747	total: 68.2ms	remaining: 1m 8s
1:	learn: 0.6905779	total: 84.9ms	remaining: 42.4s
2:	learn: 0.6892876	total: 91.9ms	remaining: 30.5s
3:	learn: 0.6880039	total: 105ms	remaining: 26.2s
4:	learn: 0.6867263	total: 113ms	remaining: 22.5s
5:	learn: 0.6854551	total: 118ms	remaining: 19.6s
6:	learn: 0.6840831	total: 192ms	remaining: 27.2s
7:	learn: 0.6828492	total: 268ms	remaining: 33.2s
8:	learn: 0.6815965	total: 283ms	remaining: 31.1s
9:	learn: 0.6803569	total: 302ms	remaining: 29.9s
10:	learn: 0.6791147	total: 316ms	remaining: 28.4s
11:	learn: 0.6778885	total: 356ms	remaining: 29.3s
12:	learn: 0.6766639	total: 374ms	remaining: 28.4s
13:	learn: 0.6754670	total: 457ms	remaining: 32.2s
14:	learn: 0.6742510	total: 476ms	remaining: 31.2s
15:	learn: 0.6730454	total: 492ms	remaining: 30.3s
16:	learn: 0.6718409	total: 510ms	remaining: 29.5s
17:	learn: 0.6706632	total: 611ms	remaining: 33.3s
18:	learn: 0.6694806	total: 668ms	remaining: 34.5s
19:	learn: 0.6683066	total: 740ms	rema

In [157]:
from sklearn.metrics import precision_score, f1_score, recall_score

y_predict = model1.predict(X_test)

print('precision ', precision_score(Y_test, y_predict, average='samples'))
print('recall ', recall_score(Y_test, y_predict, average='macro'))
print('f1 ', f1_score(Y_test, y_predict, average='samples'))

precision  0.35789473684210527
recall  0.16790123456790124
f1  0.30999999999999994


c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))

In [150]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])



c:\Users\dsfs-\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [149]:
params = {"iterations": 100,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False}

scores = cross_val_score(
    pool, 
    params
    scoring='f1_samples',
    cv=3,
    n_jobs=1
)

TypeError: cv() got an unexpected keyword argument 'n_jobs'